# Command-line Contra

### Aim:
Draw board in command-line type visualisation which can show beginning and end of contra dance move defined by user.

### Notes:
* I will be using camelCase for functions and variable.
* Position variables are zero indexed and the y-axis is top to bottom.
* Top of the page is up

****

### Thoughts
* I need relative position on the dance floor as well as relative position in the set
* Do I want to make the lists of the entities on the dance floor into sets?
* Looks like definitions of moves will go into 'Sets' and 'Partners' classes

In [2]:
# imports and setup

import numpy as np
from pprint import pprint as pp

In [140]:
# define classes

class DanceFloor(object):
    """A schematic of a dance floor.
    
    Attributes:
        formation: The formation of the dancers (e.g. proper, improper, becket)
        length: An integer giving the length of the dance floor.
        width: An integer giving the width of the dance floor.
        floor: An array representing the dance floor.
        middle: A float giving the x-coordinate of the centre line of the dance floor.
        dancers: A dictionary with key-value pairs of dancers on the floor and their positions.
        couples: A list of all couples on the dance floor.
        neighbours: A list of all neighbours on the dance floor.
        opposites: A list of all opposites on the dance floor
        ones: A list of all the first couples on the dance floor.
        twos: A list of all the second couples on the dance floor.
        sets: A list of all sets on the dance floor.
        
    """
    
    def __init__(self, formation, length, width=8):
        self.formation = formation.lower()
        self.length = length
        self.width = width
        try:
            self.floor = np.full((length, width), "_", dtype=str)
        except TypeError:
            print("Dimension argument(s) for DanceFloor object must be integer(s).")
        self.middle = (width-1)/2.0
        self.dancers = dict()
        self.couples = Pairs(self.floor)
        self.neighbours = Pairs(self.floor)
        self.opposites = Pairs(self.floor)
        self.ones = Pairs(self.floor)
        self.twos = Pairs(self.floor)
        self.sets = Sets(self.floor)
        
    
    def updateFloor(self):
        """Updates dancefloor after move"""
        # reset floor
        self.floor = np.full((self.length, self.width), "_", dtype=str)
        # update dancers' symbols on floor
        for dancer in self.dancers:
            self.floor[self.dancers[dancer].position[1], self.dancers[dancer].position[0]] = dancer
        
        
    def showFloor(self, title=None):
        """Shows the current state of the dance floor with optional title"""
        if title != None:
            print(title)
        # draw floor
        for i in self.floor:
            print(i)
        print("")
        
        
    def listDancers(self):
        """Lists the dancers currently on the floor with their positions"""
        for index in self.dancers:
            print(index + " ~ " + str(self.dancers[index].position))
            
            
    def addDancer(self, dancer):
        """Adds a dancer to the floor at their starting position"""
        # check symbol not being used by another dancer
        if dancer.symbol not in self.dancers:
            # check symbol is single character
            if len(dancer.symbol) == 1:
                try:               
                    # check for another dancer in the same position
                    if self.floor[dancer.position[1], dancer.position[0]] == "_":
                        # change floor marker to dancer's symbol and update dict of dancers
                        self.dancers[dancer.symbol] = dancer
                        self.floor[dancer.position[1], dancer.position[0]] = dancer.symbol
                    else:
                        print("There is already a dancer at the position %s." % dancer.position)
                except IndexError:
                    print("The position %s is out of bounds." % dancer.position)
            else:
                print("Symbol for a dancer must be a single character.")
        else:
            print("The symbol '%s' is already being used for a dancer." % dancer.symbol)
                
                
class Dancer(object):
    """ A sprite representing a dancer.
    
    Attributes:
        symbol: A character representing the dancer on the floor e.g. 'a', '1'.
        floor: The dance floor on which this dancer are dancing.
        position: A two-integer array giving the dancer's position e.g. [2,4].
        partner: The dancer who is this dancer's partner.
        neighbour: The dancer who is this dancer's neighbour.
    
    """
    def __init__(self, symbol, position, floor): 
        self.symbol = symbol
        self.floor = floor
        self.position = position
        self.partner = None
        self.neighbour = None
        self.floor.addDancer(self)  # add dancer to dance floor    
        
        
class Couple(object):
    """
    Two dancers who interact as a couple in a dance.
    
    Attributes:
        floor: The dance floor on which the couple are dancing
        d1: first dancer in the couple ('lead' place)
        d2: second dancer in couple ('follow' place)
    
    """
    def __init__(self, floor, d1sym, d1pos, d2sym, d2pos):
        self.floor = floor
        self.d1 = Dancer(d1sym, d1pos, floor)
        self.d2 = Dancer(d2sym, d2pos, floor)        
        self.d1.partner = self.d2
        self.d2.partner = self.d1
        self.floor.couples.append(self)
        
        
class Pair(object):
    """Two dancers who interact as a pair in a dance.
    
    Attributes:
        floor: The dance floor on which the pair is dancing.
        d1: One of the pair.
        d2: The other of the pair.
        
    """
    def __init__(self, floor, d1, d2):
        self.floor = floor
        self.d1 = d1
        self.d2 = d2
        
class Neighbour(Pair):
    """Two dancers who interact as neighbours"""
    def __init__(self, floor, d1, d2):
        Pair.__init__(self, floor, d1, d2)
        self.floor.neighbours.append(self)
        

class Opposite(Pair):
    """Two dancers, a follow from one couple and the lead from the other"""
    def __init__(self, floor, d1, d2):
        Pair.__init__(self, floor, d1, d2)
        self.floor.opposites.append(self)     
            
class Set(object):
    """A number of dancers (currently set at four) who interact as a set.
    
    Attributes:
        floor: The floor on which the set are dancing.
        formation: The formation of the dance (e.g. proper, improper, becket).
        ones: A Couple object representing first couple.
        twos: A Couple object representing second couple.
    
    """        
        
    def __init__(self, floor, ones, twos):
        self.floor = floor
        self.formation = self.floor.formation
        self.ones = ones
        self.twos = twos
        # add couples to floor's list of ones and twos
        self.floor.ones.append(self.ones)
        self.floor.twos.append(self.twos)
        
        # define neighbours, opposites, and set positions
        if self.formation=="proper":
            self.neighbour1 = Neighbour(self.floor, self.ones.d1, self.twos.d1)
            self.neighbour2 = Neighbour(self.floor, self.ones.d2, self.twos.d2)
            self.opposite1 = Opposite(self.floor, self.ones.d1, self.twos.d2)
            self.opposite2 = Opposite(self.floor, self.twos.d1, self.ones.d2)
            self.positionNums, self.positionLocs = assignPositions(
                self.ones.d1, self.ones.d2, self.twos.d2, self.twos.d1)
        else:
            print("No such formation.")
            exit(2)
            
        # add set to list of sets
        self.floor.sets.append(self)
        

    def assignPositions(*dancers):
        """Creates two dictionaries: one giving the position number for a location, and the other doing the reverse.
        
        Arguments are dancers in the set. They must be given in order going round the circle clockwise, i.e.
        1   2
        4   3
        
        """
        positionNums = dict()
        positionLocs = dict()
        
        # create number->location dict, iterating through arguments
        for i in range(len(dancers)):
            positionNums[i+1] = dancers[i].position
            
        # create location->number dict, iterating though arguments. Note location is tuple, not list.
        for i in range(len(dancers)):
            positionLocs[tuple(dancers[i].position)] = i+1
            
        return (positionNums, positionLocs)
        
        
class Pairs(object):
    """Pairs of dancers (e.g. neighbours, partners) who interact in the dance.
    
    Attributes:
        floor: The dance floor on which the pairs are dancing.
        pairs: A list of the pairs dancing
        
    """
    
    def __init__(self, floor):
        self.floor = floor
        self.pairs = []
        
        
    def append(self, pair):
        self.pairs.append(pair)
        
        
    def swapPlaces(self):
        """Move ending with the pair in each other's places"""
        for pair in self.pairs:
            pair.d1.position, pair.d2.position = pair.d2.position, pair.d1.position
        floor.updateFloor()
        
        
    def swing(self, facing="up"):
        """Couples swing, ending with follow on the right of the lead and couple facing up, down, left, or right"""
        # define ending position assuming lead stays in place, may change later
        # also need to add in facing out and in
        for pair in self.pairs:
            if facing=='up':
                pair.d2.position = np.array(pair.d1.position) + [1,0]
            if facing=='down':
                pair.d2.position = np.array(pair.d1.position) - [1,0]
            if facing=='right':
                pair.d2.position = np.array(pair.d1.position) + [0,1]
            if facing=='left':
                pair.d2.position = np.array(pair.d1.position) - [0,1]
        floor.updateFloor()
                        
            
class Sets(object):
    """A list of all sets on the dance floor
    
    Attributes:
        floor: The floor on which the sets are dancing.
        sets: A list of all sets on the dance floor.
        
    """
    
    def __init__(self, floor):
        self.floor = floor
        self.sets = []
        
        
    def append(self, sett):  # to avoid clashing with 'set'
        self.sets.append(sett)

### Moves involving one set only
Start by defining:
* Partners
    * [x] Swap places
    * [ ] Swing
* Sets
    * [ ] Forward and back
    * [ ] Ladies' chain
    * [ ] Rights and lefts
    * [ ] Down the hall and back
    * [ ] Petronella

In [146]:
# set up objects and assign convenience names
floor = DanceFloor('proper', 8)
ones = floor.ones
twos = floor.twos
couples= floor.couples
neighbours = floor.neighbours
opposites = floor.opposites
#floor.showFloor("Empty floor")

myOnes = Couple(floor, 'A', [2,3], 'a', [5,3])
myTwos = Couple(floor, 'B', [2,4], 'b', [5,4])
mySet = Set(floor, myOnes, myTwos)
floor.showFloor("Set in starting positions")

# define dance moves

#### different couples swap - this all works just fine
ones.swapPlaces()
# floor.showFloor("Ones swap")
# twos.swapPlaces()
# floor.showFloor("Twos swap")
#neighbours.swapPlaces()
#floor.showFloor("Neighbours swap")
opposites.swapPlaces()
floor.showFloor("Opposites swap")
# couples.swapPlaces()
# floor.showFloor("All couples swap")

#### swing
couples.swing()
floor.showFloor()

Set in starting positions
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'A' '_' '_' 'a' '_' '_']
['_' '_' 'B' '_' '_' 'b' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']

Opposites swap
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'B' '_' '_' 'b' '_' '_']
['_' '_' 'a' '_' '_' 'A' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']

['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' 'B' 'b' '_' '_' '_' '_']
['_' '_' '_' '_' '_' 'A' 'a' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']



In [118]:
[0,2]+[0,1]


[0, 2, 0, 1]